In [0]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAppName('appName').setMaster('local[4]').set('spark.jars.packages', 'graphframes:graphframes:0.8.0-spark2.4-s_2.11')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-954607068907094> in <module>
----> 1 import findspark
      2 findspark.init()
      3 findspark.find()
      4 import pyspark
      5 findspark.find()

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    165             # Import the desired module. If you’re seeing this while debugging a failed import,
    166             # look at preceding stack frames for relevant error information.
--> 167             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    168 
    169             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'findspark'

# Utilización del servicio de alquiler de bicicletas en Toronto en el año 2018

### Disponible en Kaggle en:
https://www.kaggle.com/jackywang529/toronto-bikeshare-data


El propósito de este análisis es utilizar los conjuntos de datos trimestrales del año 2018 de la empresa de alquiler de bicicletas en Toronto. Se trata de *cuatro* conjuntos de datos separados, que incluyen entre 178.559 y 822.536 observaciones, siempre con nueve variables. Cada fila representa un viaje realizado.

### Variables y significado

Las variables utilizadas para describir cada viaje son:

* trip_id – identificador global del viaje
* trip_duration_seconds – duración del viaje en segundos
* from_station_id – identificador numérico de la estación de origen
* trip_start_time – instante (timestamp) en el que se inició el viaje
* from_station_name – nombre de la intersección más cercana a la estación origen
* trip_stop_time – instante (timestamp) en el que finalizó el viaje
* to_station_id – identificador numérico de la estación de destino
* to_station_name – nombre de la intersección más cercana a la estación de destino
* user_type – tipo de usuario (indicador binario): miembro registrado con cuota anual / usuario ocasional no registrado

**Nombre completo del alumno:**

**INSTRUCCIONES**: en cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). **No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

### Sobre los cuatro datasets anteriores (Bike Share Toronto Ridership_Q1 2018.csv hasta Q4) se pide:

**(1 punto)** Ejercicio 1

* Leer por separado cada uno de ellos (sin cachear), tratando de que Spark infiera el tipo de dato de cada columna, y **unirlos en un solo DF** que tampoco debe ser cacheada todavía, ya que en el siguiente paso aún realizaremos otro pre-procesamiento.
* Los cuatro contienen las mismas columnas por lo que no habrá problemas para utilizar la operación `union` encadenada tres veces para crear el DF final.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
tripsQ1 = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/Bike_Share_Toronto_Ridership_Q1_2018.csv")
tripsQ2 = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/Bike_Share_Toronto_Ridership_Q2_2018.csv")
tripsQ3 = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/Bike_Share_Toronto_Ridership_Q3_2018.csv")
tripsQ4 = spark.read.option("header", "true").option("inferSchema", "true").csv("/FileStore/tables/Bike_Share_Toronto_Ridership_Q4_2018.csv")
tripsTorontoRawDF = tripsQ1.union(tripsQ2).union(tripsQ3).union(tripsQ4)
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
# YOUR CODE HERE



Out[17]: DataFrame[key: string, value: string]

In [0]:
from pyspark.sql.types import DoubleType
assert(tripsTorontoRawDF.count() == 1922955)

**(1 punto)** Ejercicio 2

* Las columnas `trip_start_time` y `trip_stop_time` son en realidad instantes de tiempo que Spark debería procesar como timestamp. Reemplaza **ambas columnas** por su versión convertida a timestamp, utilizando `withColumn` y donde el nuevo valor de la columna viene dado por el siguiente código:
        F.from_unixtime(F.unix_timestamp('nombreColumna', 'MM/dd/yyyy HH:mm')).cast("timestamp"))
El DF resultante debe ser almacenado en la variable `tripsTorontoDF`.

In [0]:
# No olvides los imports que necesites...
from pyspark.sql import functions as F
# LÍNEAS EVALUABLES, NO RENOMBRAR LAS VARIABLES
tripsTorontoDF = tripsTorontoRawDF.withColumn("trip_start_time", F.from_unixtime(F.unix_timestamp("trip_start_time", 'MM/dd/yyyy HH:mm')).cast("timestamp"))\
                                  .withColumn("trip_stop_time", F.from_unixtime(F.unix_timestamp("trip_stop_time", 'MM/dd/yyyy HH:mm')).cast("timestamp"))
tripsTorontoDF.printSchema()
tripsTorontoDF.select("trip_start_time").show()

# YOUR CODE HERE

root
 |-- trip_id: integer (nullable = true)
 |-- trip_duration_seconds: integer (nullable = true)
 |-- from_station_id: integer (nullable = true)
 |-- trip_start_time: timestamp (nullable = true)
 |-- from_station_name: string (nullable = true)
 |-- trip_stop_time: timestamp (nullable = true)
 |-- to_station_id: integer (nullable = true)
 |-- to_station_name: string (nullable = true)
 |-- user_type: string (nullable = true)

+-------------------+
|    trip_start_time|
+-------------------+
|2018-01-01 00:47:00|
|2018-01-01 00:52:00|
|2018-01-01 00:55:00|
|2018-01-01 00:57:00|
|2018-01-01 01:00:00|
|2018-01-01 01:07:00|
|2018-01-01 01:33:00|
|2018-01-01 01:34:00|
|2018-01-01 01:37:00|
|2018-01-01 01:38:00|
|2018-01-01 01:39:00|
|2018-01-01 01:39:00|
|2018-01-01 01:49:00|
|2018-01-01 01:57:00|
|2018-01-01 02:06:00|
|2018-01-01 02:25:00|
|2018-01-01 02:29:00|
|2018-01-01 02:30:00|
|2018-01-01 02:45:00|
|2018-01-01 02:53:00|
+-------------------+
only showing top 20 rows



In [0]:
typesDict = dict(tripsTorontoDF.dtypes)
assert(typesDict["trip_start_time"] == "timestamp") 
assert(typesDict["trip_stop_time"] == "timestamp") 

**(1 punto)** Ejercicio 3

Partiendo de `tripsTorontoDF`, realizar las siguientes transformaciones encadenadas en este orden para crear un nuevo DF:
* Primero, debemos quedarnos solamente con las filas donde `trip_start_time` no sea null.
* Sobre el DF resultado de lo anterior, añadir una columna adicional **Mes** y con el mes representado en **trip_start_time**. Dicha columna será de tipo entero y se puede obtener usando `withColumn` con la función `F.month("colName")`, que recibe un nombre de columna y devuelve un objeto columna de enteros que van de 1 a 12. 
* Encadenar esta transformación con otra en la que la columna **Mes** sea reemplazada por su traducción a  cadena de caracteres de 3 letras, siendo la correspondencia 1: Ene, 2: Feb, 3: Mar, 4: Abr, 5: May, 6: Jun, 7: Jul, 8: Ago, 9: Sep, 10: Oct, 11: Nov, 12: Dic.
* Finalmente, añadir una nueva columna **Hora** que contenga la hora de inicio del viaje, aplicando `withColumn` con la función `F.hour("colName")` que recibe un nombre de columna y recibe un objeto columna de enteros de 0 a 23.
* El DF resultante de todas estas transformaciones debe guardarse en la variable `tripsTorontoTimesDF`, que por tanto tendrá 2 columnas más que el DF original `tripsTorontoDF`, y que debe quedar **cacheado**.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
from pyspark.sql.types import IntegerType
tripsTorontoTimesDF = tripsTorontoDF.filter("trip_start_time is not null")
tripsTorontoTimesDF = tripsTorontoTimesDF.withColumn("Mes", F.month("trip_start_time").cast(IntegerType()))\
                                         .withColumn("Mes", F.when(F.col("Mes") == "1", "Ene")\
                                                             .when(F.col("Mes") == "2", "Feb")\
                                                             .when(F.col("Mes") == "3", "Mar")\
                                                             .when(F.col("Mes") == "4", "Abr")\
                                                             .when(F.col("Mes") == "5", "May")\
                                                             .when(F.col("Mes") == "6", "Jun")\
                                                             .when(F.col("Mes") == "7", "Jul")\
                                                             .when(F.col("Mes") == "8", "Ago")\
                                                             .when(F.col("Mes") == "9", "Sep")\
                                                             .when(F.col("Mes") == "10", "Oct")\
                                                             .when(F.col("Mes") == "11", "Nov")\
                                                             .when(F.col("Mes") == "12", "Dic"))\
                                          .withColumn("Hora", F.hour("trip_start_time"))
                                                     
tripsTorontoTimesDF.cache()

Out[19]: DataFrame[trip_id: int, trip_duration_seconds: int, from_station_id: int, trip_start_time: timestamp, from_station_name: string, trip_stop_time: timestamp, to_station_id: int, to_station_name: string, user_type: string, Mes: string, Hora: int]

In [0]:
tripsPerMonth = tripsTorontoTimesDF.groupBy("Mes").count().sort("Mes").collect()
assert(tripsPerMonth[0]["count"] == 94783)
assert(tripsPerMonth[1]["count"] == 281219)
assert(tripsPerMonth[2]["count"] == 83324)
assert(tripsPerMonth[3]["count"] == 43859)
assert(tripsPerMonth[4]["count"] == 49731)
assert(tripsPerMonth[5]["count"] == 286316)
assert(tripsPerMonth[6]["count"] == 250837)
assert((tripsPerMonth[7]["count"] == 84959) | (tripsPerMonth[7]["count"] == 84969))
assert(tripsPerMonth[8]["count"] == 212750)
assert(tripsPerMonth[9]["count"] == 104287)
assert(tripsPerMonth[10]["count"] == 175879)
assert(tripsPerMonth[11]["count"] == 255001)

**(1 punto)** Ejercicio 4

* Partiendo de `tripsTorontoTimesDF`, crear un nuevo DataFrame con **tantas filas como horas tiene el día, y tantas columnas como meses del año** de manera que cada celda indique el **número de viajes** que comenzaron a esa hora en ese mes del año. Guardar el resultado en la variable `tripsPerMonthAndHourDF`, cuyas filas deben quedar ordenadas en base a la hora (de 0 a 23), y cuyas columnas deben estar también ordenadas desde `"Ene"` a `"Dic"`, con `"Hora"` como primera columna.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
tripsPerMonthAndHourDF = tripsTorontoTimesDF.groupBy("Hora").pivot("Mes").agg(
                            F.count("trip_id").alias("Cantidad de viajes")).select("Hora", "Ene","Feb", "Mar", "Abr","May", "Jun","Jul", "Ago", "Sep", "Oct", "Nov", "Dic")\
                             .orderBy("Hora", ascending =True)
                            
tripsPerMonthAndHourDF.show()

# YOUR CODE HERE

+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+
|Hora| Ene| Feb|  Mar|  Abr|  May|  Jun|  Jul|  Ago|  Sep|  Oct|  Nov| Dic|
+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+
|   0| 425| 412|  689|  770| 1913| 2859| 3418| 2912| 2623| 1419|  888| 782|
|   1| 266| 308|  434|  493| 1180| 1725| 1991| 1831| 1689|  943|  637| 525|
|   2| 201| 183|  318|  374|  804| 1274| 1356| 1360| 1344|  684|  442| 417|
|   3|  91| 101|  156|  130|  361|  587|  646|  595|  632|  245|  211| 164|
|   4|  49|  56|  133|  114|  227|  463|  494|  504|  467|  264|  253| 187|
|   5| 198| 226|  348|  381|  627|  877| 1121| 1110| 1032|  860|  569| 373|
|   6| 603| 668|  995| 1092| 2043| 2802| 3232| 3088| 2800| 2373| 1745|1082|
|   7|1742|2011| 3303| 3436| 6626| 8209| 8978| 8146| 8130| 6217| 4286|2914|
|   8|5001|5199| 8682| 9825|17855|20602|23931|22475|21216|17825|11720|8293|
|   9|3728|3769| 6497| 6790|12063|14044|16089|14978|14710|12381| 8314|6287|
|  10|1975|2

In [0]:
assert(len(tripsPerMonthAndHourDF.columns) == 13)
assert(tripsPerMonthAndHourDF.columns[0] == "Hora")
assert(tripsPerMonthAndHourDF.columns[12] == "Dic")
assert(tripsPerMonthAndHourDF.count() == 24)
todasHoras = tripsPerMonthAndHourDF.collect()
assert((todasHoras[0]["Hora"] == 0) & (todasHoras[0]["Dic"]==782))
assert((todasHoras[23]["Hora"] == 23) & (todasHoras[23]["Dic"]==1208))

**(3 puntos)** Ejercicio 5. 

Partiendo de `tripsTorontoTimesDF` definido anteriormente, añadir las siguientes columnas:

* Primero, tres columnas adicionales llamadas `dur_media`, `dur_min`, `dur_max` que contengan, respectivamente, **la duración media, mínima y máxima de los viajes que parten de esa misma estación de origen (from_station_id) a esa misma hora y en ese mismo mes del año**. Es decir, queremos una columna extra para que podamos tener, junto a cada viaje, información agregada de los viajes similares, entendidos como aquellos que salieron a la misma hora de la misma estación. **No se debe utilizar JOIN sino solo funciones de ventana**.
* A continuación, otra columna adicional `diff_dur_porc` que contenga la diferencia, medida en porcentaje, entre la duración del viaje y la duración media de los viajes similares calculada en el apartado anterior. Dicha diferencia debe calcularse como la resta de la duración del viaje menos la duración media, dividida entre la duración media y multiplicada por 100. El resultado debe obtenerse aplicando operaciones aritméticas con columnas existentes, **sin utilizar `when`**.
* El DF resultante con las 4 columnas nuevas que hemos añadido debe almacenarse en la variable `tripsTorontoExtraInfoDF`.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
from pyspark.sql import Window 
windowHoraMesEstacion = Window().partitionBy("from_station_id", "Hora", "Mes")
tripsTorontoExtraInfoDF = tripsTorontoTimesDF.withColumn("dur_media", F.mean("trip_duration_seconds").over(windowHoraMesEstacion))\
                                             .withColumn("dur_min", F.min("trip_duration_seconds").over(windowHoraMesEstacion))\
                                             .withColumn("dur_max", F.max("trip_duration_seconds").over(windowHoraMesEstacion))\
                                             .withColumn("diff_dur_porc", ((F.col("trip_duration_seconds")-F.col("dur_media"))/(F.col("dur_media")))*100)

tripsTorontoExtraInfoDF.select("trip_duration_seconds", "Hora", "Mes", "dur_media", "dur_min", "dur_max", "diff_dur_porc").show()
# YOUR CODE HERE


+---------------------+----+---+------------------+-------+-------+-------------------+
|trip_duration_seconds|Hora|Mes|         dur_media|dur_min|dur_max|      diff_dur_porc|
+---------------------+----+---+------------------+-------+-------+-------------------+
|                 4951|   1|May|1938.6666666666667|    245|   4951| 155.38170563961484|
|                 4912|   1|May|1938.6666666666667|    245|   4951| 153.37001375515814|
|                  313|   1|May|1938.6666666666667|    245|   4951| -83.85488308115544|
|                  502|   1|May|1938.6666666666667|    245|   4951| -74.10591471801926|
|                  245|   1|May|1938.6666666666667|    245|   4951| -87.36244841815682|
|                  709|   1|May|1938.6666666666667|    245|   4951|-63.428473177441546|
|                  410|   3|Oct|             410.0|    410|    410|                0.0|
|                  562|   4|Ene|             562.0|    562|    562|                0.0|
|                 1904|   6|Jul|

In [0]:
r = tripsTorontoExtraInfoDF.where("trip_id = '2970611'").head()
assert(r.dur_media - 783.366666667 < 0.001)
assert(r.diff_dur_porc - 44.24918088591975 < 0.001)
assert(r.dur_min == 167)
assert(r.dur_max == 2333)

**(3 puntos)** Ejercicio 6

* Partiendo de `tripsTorontoTimesDF`, crear un **grafo** llamado `bikeGraph` utilizando como identificador de los vértices los identificadores de las estaciones. Construye primero un DF con todos los identificadores de las estaciones, simplemente seleccionando **from_station_id**, renombrando adecuadamente el nombre de columna. Puedes almacenar este DF en la variable `verticesDF`. También tendrás que renombrar las columnas **from_station_id** y **to_station_id** en el DF de aristas, para el que además deberás seleccionar solo dichas columnas y quitar las filas repetidas ya que solo necesitamos considerar una vez cada ruta (cada pareja de estación inicial y final). Puedes almacenar el resultado del renombramiento y la eliminación de repetidos en la variable `edgesDF`.
* Una vez creado, aplica el algoritmo `pageRank` pasando como único parámetro `maxIter = 5`. El algoritmo puede llegar a emplear más de 10 minutos. 
* Almacena el grafo devuelto por dicha función en la variable `pageRankGraph`, recupera el DF de sus vértices, ordénalo descendentemente en base a la columna `pagerank` y almacena el resultado en la variable `sortedPageRankGraphVerticesDF`
* Obtén el identificador de la estación más relevante (con mayor valor de la métrica pageRank, que ocupará la primera fila tras la ordenación), y almacena dicho identificador en la variable `id_mas_relevante`.
* Crea un nuevo DF de una sola fila y tres columnas llamadas `dur_media`, `dur_min` y `dur_max` con la duración **media, mínima y máxima** de los viajes de `tripsTorontoTimesDF` que **empiezan** en dicha estación (sin tener en cuenta distinción de horas o meses). **No debe usarse la función `withColumn` sino crear las columnas al vuelo con `select`**. Debe quedar almacenado en la variable `durEstMasRelevantesDF`

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# Descomentar la siguiente línea antes de lanzar pageRank:
spark.sparkContext.setCheckpointDir("/tmp")
from graphframes import GraphFrame
verticesDF = tripsTorontoTimesDF.select(F.col("from_station_id").alias("id"))\
                                .distinct().cache()
edgesDF = tripsTorontoTimesDF.withColumnRenamed("from_station_id", "src")\
                             .withColumnRenamed("to_station_id", "dst")\
                             .select("src", "dst")\
                             .distinct()\
                             .cache()
bikeGraph = GraphFrame(verticesDF, edgesDF)
pageRankGraph = bikegraph.pageRank(maxIter = 5)
sortedPageRankGraphVerticesDF = pageRankGraph.vertices.orderBy(F.col("pagerank").desc())
id_mas_relevante = sortedPageRankGraphVerticesDF.select("id").first()[0]
durEstMasRelevantesDF = tripsTorontoTimesDF.where(F.col("from_station_id") == id_mas_relevante)\
                                           .select(F.mean(F.col("trip_duration_seconds")).alias("dur_media"))\
                                           .select(F.min(F.col("trip_duration_seconds")).alias("dur_min"))\
                                           .select(F.max(F.col("trip_duration_seconds")).alias("dus_max")) 

# YOUR CODE HERE

UsageError: Line magic function `%%sh` not found.


In [0]:
assert(sortedPageRankGraphVerticesDF.head()["pagerank"] - 1.4427 < 0.01)
assert(durEstMasRelevantesDF.count() == 1)
assert(len(durEstMasRelevantesDF.columns) == 3)
rEstMasRelevantes = durEstMasRelevantesDF.head()
assert(rEstMasRelevantes.dur_min == 61)
assert(id_mas_relevante == 7060)
assert(rEstMasRelevantes.dur_media - 747.6957692082626 < 0.001)
assert(rEstMasRelevantes.dur_max == 35130)